# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import matplotlib.pyplot as plt #The matplotlib visuailzation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.240427,
 'week52high': 472.65,
 'week52low': 205.46,
 'marketcap': 1945947568277,
 'employees': 140348,
 'day200MovingAvg': 313.45,
 'day50MovingAvg': 382,
 'float': 4327143464,
 'avg10Volume': 48310659.7,
 'avg30Volume': 39048936.1,
 'ttmEPS': 13.3035,
 'ttmDividendRate': 3.19,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4465605788,
 'maxChangePercent': 439.267,
 'year5ChangePercent': 2.856,
 'year2ChangePercent': 1.153,
 'year1ChangePercent': 1.248,
 'ytdChangePercent': 0.507,
 'month6ChangePercent': 0.4029,
 'month3ChangePercent': 0.4814,
 'month1ChangePercent': 0.1983,
 'day30ChangePercent': 0.2639,
 'day5ChangePercent': 0.02,
 'nextDividendDate': '2020-08-29',
 'dividendYield': 0.007091991016714177,
 'nextEarningsDate': '2020-10-20',
 'exDividendDate': '2020-07-29',
 'peRatio': 35.07,
 'beta': 1.165345391902975}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course.

In [4]:
data['year1ChangePercent']

1.248

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,102.150,0.457600,N/A
1,AAL,13.810,-0.558400,N/A
2,AAP,156.378,0.056400,N/A
3,AAPL,460.280,1.261000,N/A
4,ABBV,95.430,0.443500,N/A
...,...,...,...,...
500,YUM,91.900,-0.237170,N/A
501,ZBH,143.912,0.003371,N/A
502,ZBRA,283.580,0.351320,N/A
503,ZION,34.900,-0.204228,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,454.990,1.896747,N/A
1,DXCM,449.981,1.859113,N/A
2,AMD,85.943,1.520037,N/A
3,CARR,29.290,1.475277,N/A
4,AAPL,460.280,1.261000,N/A
5,REGN,619.750,1.037329,N/A
6,WST,280.000,0.923363,N/A
7,PYPL,207.770,0.882263,N/A
8,LRCX,381.450,0.874284,N/A
9,QRVO,136.900,0.826200,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy.

Since we've already done this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000
100000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,454.990,1.896747,4
1,DXCM,449.981,1.859113,4
2,AMD,85.943,1.520037,22
3,CARR,29.290,1.475277,66
4,AAPL,460.280,1.261000,4
5,REGN,619.750,1.037329,3
6,WST,280.000,0.923363,7
7,PYPL,207.770,0.882263,9
8,LRCX,381.450,0.874284,5
9,QRVO,136.900,0.826200,14


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe.

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100


for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])
    
hqm_dataframe

0        0.89505
1      0.0277228
2       0.546535
3       0.992079
4       0.887129
         ...    
500     0.225743
501     0.473267
502     0.837624
503     0.257426
504     0.790099
Name: One-Year Return Percentile, Length: 505, dtype: object
0       0.865347
1      0.0178218
2       0.813861
3       0.956436
4       0.643564
         ...    
500     0.423762
501     0.417822
502     0.815842
503     0.235644
504     0.784158
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.794059
1      0.926733
2      0.819802
3      0.978218
4      0.451485
         ...   
500    0.289109
501    0.556436
502    0.635644
503    0.431683
504    0.817822
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.663366
1       0.665347
2       0.730693
3       0.940594
4      0.0376238
         ...    
500     0.378218
501     0.908911
502      0.69901
503     0.477228
504     0.839604
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,101.740,N/A,0.452800,0.89505,0.184000,0.865347,0.249700,0.794059,0.097600,0.663366,N/A
1,AAL,13.975,N/A,-0.547200,0.0277228,-0.565500,0.0178218,0.376400,0.926733,0.098000,0.665347,N/A
2,AAP,158.776,N/A,0.056300,0.546535,0.144700,0.813861,0.275400,0.819802,0.114100,0.730693,N/A
3,AAPL,465.870,N/A,1.290000,0.992079,0.405100,0.956436,0.485800,0.978218,0.197200,0.940594,N/A
4,ABBV,95.390,N/A,0.446300,0.887129,0.006900,0.643564,0.106400,0.451485,-0.067000,0.0376238,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,92.300,N/A,-0.227246,0.225743,-0.118007,0.423762,0.054263,0.289109,0.051691,0.378218,N/A
501,ZBH,144.480,N/A,0.003334,0.473267,-0.119557,0.417822,0.146080,0.556436,0.175460,0.908911,N/A
502,ZBRA,293.480,N/A,0.345810,0.837624,0.145021,0.815842,0.173375,0.635644,0.105200,0.69901,N/A
503,ZION,34.100,N/A,-0.199912,0.257426,-0.261478,0.235644,0.101817,0.431683,0.066900,0.477228,N/A


## Calculating the HQM Score

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
31,AMD,84.239,N/A,1.531146,0.99604,0.628494,0.988119,0.597654,0.990099,0.590261,0.99802,0.993069
78,CARR,30.300,N/A,1.480555,0.994059,1.466883,1,0.599973,0.992079,0.215334,0.960396,0.986634
3,AAPL,465.870,N/A,1.290000,0.992079,0.405100,0.956436,0.485800,0.978218,0.197200,0.940594,0.966832
6,ABMD,314.533,N/A,0.648200,0.966337,0.692900,0.992079,0.654000,0.994059,0.176100,0.910891,0.965842
490,WST,283.000,N/A,0.947235,0.988119,0.740496,0.99604,0.378410,0.928713,0.178389,0.912871,0.956436
...,...,...,...,...,...,...,...,...,...,...,...,...
84,CCL,14.969,N/A,-0.727259,0.0019802,-0.670161,0.00594059,0.000733,0.170297,-0.074160,0.029703,0.0519802
479,WDC,38.670,N/A,-0.345458,0.108911,-0.460880,0.0514851,-0.168380,0.0158416,-0.135866,0.00594059,0.0455446
181,FE,30.380,N/A,-0.338711,0.118812,-0.442041,0.0594059,-0.275363,0.0019802,-0.283676,0.0019802,0.0455446
219,HFC,26.350,N/A,-0.518917,0.039604,-0.429381,0.0732673,-0.185291,0.0138614,-0.079429,0.0257426,0.0381188


## Selecting the 50 Best Momentum Stocks

In [13]:
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,101.740,192,0.452800,0.89505,0.184000,0.865347,0.249700,0.794059,0.097600,0.663366,0.804455
1,AAL,13.975,1403,-0.547200,0.0277228,-0.565500,0.0178218,0.376400,0.926733,0.098000,0.665347,0.409406
2,AAP,158.776,123,0.056300,0.546535,0.144700,0.813861,0.275400,0.819802,0.114100,0.730693,0.727723
3,AAPL,465.870,42,1.290000,0.992079,0.405100,0.956436,0.485800,0.978218,0.197200,0.940594,0.966832
4,ABBV,95.390,205,0.446300,0.887129,0.006900,0.643564,0.106400,0.451485,-0.067000,0.0376238,0.50495
5,ABC,103.710,189,0.196000,0.70495,0.110700,0.776238,0.151100,0.568317,0.039400,0.332673,0.595545
6,ABMD,314.533,62,0.648200,0.966337,0.692900,0.992079,0.654000,0.994059,0.176100,0.910891,0.965842
7,ABT,102.450,191,0.197200,0.706931,0.154000,0.827723,0.081000,0.366337,0.099400,0.671287,0.643069
8,ACN,235.640,83,0.231000,0.736634,0.098500,0.760396,0.247300,0.788119,0.070400,0.49505,0.69505
9,ADBE,464.390,42,0.560500,0.942574,0.234000,0.89901,0.230200,0.750495,0.000500,0.186139,0.694554


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that ormats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy: